In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
pd.set_option('display.max_rows', 100000)
import time

In [ ]:
driver = Chrome(executable_path=r"c:\Users\cm\Documents\chromedriver\chromedriver.exe")

In [23]:
driver.get("https://www.immoscout24.ch/de")

In [29]:
#Standort auswählen
where = 'Zürich'
wohnungstyp = []

In [30]:
search_form = driver.find_element(by=By.ID, value='form-field-1')
select = Select(driver.find_element(by=By.NAME, value ='s'))
for o in select.options:
    wohnungstyp.append(o.text)

In [31]:
wohnungstyp

['Wohnung, Haus',
 'Wohnung',
 'Haus',
 'Möbliertes Wohnobjekt',
 'Grundstück',
 'Parkplatz',
 'Büro, Gewerbe, Industrie',
 'Landwirtschaft',
 'Andere Objekte']

In [32]:
#Objekt von Liste oben auswählen
select.select_by_visible_text('Wohnung')
search_form.send_keys(where)

In [33]:
search_form.send_keys(Keys.ENTER)

In [34]:
driver.find_element(by=By.XPATH, value="//button[@type='submit']").click()

In [35]:
def get_listing():
    info_list = []
    for x in range(1,101):
        try:
            try:
                try:
                    element1 = driver.find_element(by=By.XPATH, value=
                                                   '//*[@id="root"]/div[2]/main/div/div[1]/div[2]/article[{}]/a/div[1]/div/div[2]/div[2]/address/button/span'.format(x)
                    )
                except:
                    element1 = driver.find_element(by=By.XPATH, value=
                                                   '//*[@id="root"]/div[2]/main/div/div[1]/div[2]/article[{}]/a/div[1]/div/div[2]/div[1]/address/button/span'.format(x)
                    )
            except:
                element1 = driver.find_element(by=By.XPATH, value=
                                               '//*[@id="root"]/div[2]/main/div/div[1]/div[2]/article[{}]/a/div[1]/div/div[2]/div[2]/address/span/span'.format(x)
                    )
            try:
                element2 = driver.find_element(by=By.XPATH, value=
                                               '//*[@id="root"]/div[2]/main/div/div[1]/div[2]/article[{}]/a/div[1]/div/div[2]/div[2]/h3'.format(x)
                )
            except:
                element2 = driver.find_element(by=By.XPATH, value=
                                               '//*[@id="root"]/div[2]/main/div/div[1]/div[2]/article[{}]/a/div[1]/div/div[2]/div[1]/h3'.format(x)
                )
        except:
            continue
        address_info = element1.text.split(',')
        listing_specification = element2.text.split(', ')
        try:
            info_list.append(
                {
                    'Address Listing': address_info[0],
                    'Postleitzahl': address_info[1],
                    'Anzahl Zimmer': listing_specification[0],
                    'Wohnfläche': listing_specification[1],
                    'Preis': listing_specification[2]
                }
            )
        except:
            try:
                info_list.append(
                    {
                        'Address Listing': address_info[0],
                        'Postleitzahl': address_info[1],
                        'Anzahl Zimmer': listing_specification[0],
                        'Preis': listing_specification[1]
                    }
                )
            except:
                info_list.append(
                    {
                        'Address Listing': address_info[0],
                        'Postleitzahl': address_info[1],
                        'Preis': listing_specification[0]
                    }
                )                
    return info_list            

In [36]:
data = get_listing()
df = pd.DataFrame(data)
#Seitenzahl bei Bedarf ändern --> immer mit 2 beginnen; letzte Seite +1
for page in range(2,18):
    driver.get("https://www.immoscout24.ch/de/wohnung/mieten/ort-zuerich?pn={}".format(page))
    df = df.append(get_listing())

In [37]:
df.reset_index(inplace=True)

In [38]:
del df['index']

In [ ]:
df.to_csv('immoscout.csv')

In [41]:
df.head()

,Address Listing,Postleitzahl,Anzahl Zimmer,Wohnfläche,Preis
0,Uetlibergstrasse 77,8045 Zürich,2 Zimmer,50 m²,CHF 2690.—
1,Uetlibergstrasse 77,8045 Zürich,1 Zimmer,37 m²,CHF 1990.—
2,Uetlibergstrasse 77,8045 Zürich,1 Zimmer,34 m²,CHF 1570.—
3,Uetlibergstrasse 77,8045 Zürich,2 Zimmer,39 m²,CHF 2490.—
4,Kürbergstrasse 24,8049 Zürich,2 Zimmer,67 m²,CHF 2950.—
